<a href="https://colab.research.google.com/github/ymann/Machine-Comprehension-Cloze-QA/blob/master/base_model_700.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Download and unzip files
!pip3 install scikit-learn
!pip3 install gensim

In [0]:
! sudo apt-get install unzip
! wget http://seas.upenn.edu/~branlin/cnn.tgz 
! tar -xzf cnn.tgz
# ! wget http://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove.6B.zip
# ! touch word2vec.txt
  
  
# Mount your google drive. 
# Use this to save your PyTorch model for submission
# from google.colab import drive
# drive.mount('/content/gdrive')
# !mkdir /content/gdrive/My\ Drive/gloves
# #Test drive access. 
# #You should have a test.txt under a new folder cis530_hw6 in your Google drive
# with open('/content/gdrive/My Drive/gloves/', 'w') as f:
#   f.write('This is a test!')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
--2019-04-27 19:30:50--  http://seas.upenn.edu/~branlin/cnn.tgz
Resolving seas.upenn.edu (seas.upenn.edu)... 158.130.68.91
Connecting to seas.upenn.edu (seas.upenn.edu)|158.130.68.91|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.seas.upenn.edu/~branlin/cnn.tgz [following]
--2019-04-27 19:30:55--  https://www.seas.upenn.edu/~branlin/cnn.tgz
Resolving www.seas.upenn.edu (www.seas.upenn.edu)... 158.130.68.91, 2607:f470:8:64:5ea5::9
Connecting to www.seas.upenn.edu (www.seas.upenn.edu)|158.130.68.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217447312 (207M) 

In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
  
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

    100% |████████████████████████████████| 614.8MB 24kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [0]:
!git clone https://github.com/cis700/hw1-release.git
!mv hw1-release/helper.py .
!rm -rf hw1-release/
!rm -rf hw1/

Cloning into 'hw1-release'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 26 (delta 6), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://97c4320e.ngrok.io


In [0]:
import torch.nn as nn
from torch.utils.data import DataLoader
import os
from collections import Counter
from nltk import ngrams
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phraser, Phrases
from gensim.test.utils import common_texts, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec, get_glove_info
import numpy as np
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [0]:
# %%writefile questions.py
import torch
import os

class QuestionDataset(torch.utils.data.Dataset):
  
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.files = os.listdir(os.path.join(os.getcwd(), root_dir))
    
  def __len__(self):
    return len(self.files)
  
  def __getitem__(self, idx):
    if idx % 10000 == 0:
      print (idx)
    with open(os.path.join(self.root_dir, self.files[idx]), 'r') as f:
      s = f.read()
    
    lines = s.split('\n')
    doc = lines[2]
    tokens = doc.split()
    query = lines[4]
    query_tokens = query.split()
    answer = lines[6]
#     ent = [tuple(i.split(":")) for i in lines[8:]]
#     entities = {i:j for i,j in ent}
    return (tokens, query_tokens, answer)

train_set = QuestionDataset("cnn/questions/training")
test_set = QuestionDataset("cnn/questions/test")

In [0]:
train_set = QuestionDataset("cnn/questions/training")
test_set = QuestionDataset("cnn/questions/test")

In [0]:
# model.get_vector('ryan')
# get_glove_info(glove_file)
len(test_set)

3198

In [0]:
# print(train_set[0][0])
glove_file = datapath('/content/glove.6B.50d.txt')
tmp_file = get_tmpfile("./word2vec.txt")
glove2word2vec(glove_file, tmp_file)
em = KeyedVectors.load_word2vec_format(tmp_file)  

FileNotFoundError: ignored

In [0]:
%%writefile BiLSTM.py

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack

class BiLSTM(nn.Module):
    def __init__(self, emsize, hidden_dim, vocab_size, num_entities, batch_size, weights_matrix, dropout=0.2):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        super(BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.emsize = emsize
        self.batch_size = batch_size
        self.hidden_init = (torch.zeros(2, self.batch_size, self.hidden_dim, requires_grad=True).to(device)), (torch.zeros(2, self.batch_size, self.hidden_dim, requires_grad=True).to(device))
        self.dropout = nn.Dropout(dropout)
        self.start_symbols = torch.ones(10, 1).long().to(device)
        self.fc1 = nn.Linear(hidden_dim*2, num_entities)
        # Vocabulary embedding
        self.embedding = nn.Embedding(vocab_size, emsize)
        self.embedding.load_state_dict({'weight': weights_matrix})
        self.embedding.weight.requires_grad = False
        # RNN Encoder and Decoder
        self.lstm = nn.LSTM(input_size=emsize,
                               hidden_size=hidden_dim,
                               dropout=dropout,
                               batch_first=True,
                               bidirectional=True)
        
    def forward(self, dq):
      padded = rnn_utils.pad_sequence(dq).transpose(1, 0)
      lengths = torch.Tensor([len(c) for c in dq])
      embedded = self.embedding(padded)
     #pack sequence
      packed_embed = pack(embedded, lengths, batch_first=True)
      packed_output, (hidden, cell) = self.lstm(packed_embed)

      #unpack sequence
      output, input_sizes = unpack(packed_output, batch_first=True)
        
      final_output = output[torch.arange(lengths.size()[0], out=torch.LongTensor()), lengths.long() - 1]
      return self.fc1(final_output)
      

In [0]:
# # Computing vocabulary size
# words = set()
# entities = set()
# word_dict = {}
# curr_index = 2
# for d, q, _ in train_set:
#   thing = set(d + q)
#   entity_things = {i for i in thing if i.startswith('@entity')}
#   words = words.union(thing)
#   entities = entities.union(entity_things)
#   for i in words:
#     if i not in word_dict:
#       word_dict[i] = curr_index
#       curr_index += 1
#   del thing
#   del entity_things
# for d, q, _ in test_set:
#   thing = set(d + q)
#   entity_things = {i for i in thing if i.startswith('@entity')}
#   words.union(thing)
#   entities = entities.union(entity_things)
#   for i in words:
#     if i not in word_dict:
#       word_dict[i] = curr_index
#       curr_index += 1
#   del thing
#   del entity_things
# print(len(words))
# print(len(entities))

with open('vocab.txt', 'a+') as f:
  for i, j in word_dict.items():
    f.write(i + "\t" + str(j) + "\n")
# num_vocab = set().union(*[set(d + q) for d, q, _ in train_set]).size()

In [0]:
for d, q, _ in test_set:
  thing = set(d + q)
  words.union(*thing)
  del thing
print(len(words))

In [0]:
def create_matrix(vocab_size, vocab_file, model, embedding_size=50):
    with open(vocab_file, 'r') as f:
        s = f.readlines()
    vocab_dict = {i.split('\t')[0]: int(i.split('\t')[1]) for i in s
    embedding_matrix = np.zeros((vocab_size, embedding_size))
    for i, j in vocab_dict.items():
        try:
            embedding_matrix[j] = np.array(model.get_vector(i))
        except KeyError:
            embedding_matrix[j] = np.random.normal(0, 1, embedding_size)
    return torch.Tensor(embedding_matrix).to(device)

In [0]:
emsize = 50
hidden_dim = 75
vocab_size = 118497 + 1 # for the delimiter
num_entities = 584
batch_size = 50
num_epochs = 5

In [0]:
%%writefile foo.py

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
from collections import Counter
from nltk import ngrams
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phraser, Phrases
from gensim.test.utils import common_texts, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec, get_glove_info
import numpy as np
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
from questions import train_set, test_set
from BiLSTM import BiLSTM
import datetime
from helper import Logger

emsize = 50
hidden_dim = 75
vocab_size = 118497 + 1 # for the delimiter
num_entities = 584
batch_size = 50
num_epochs = 5

logger = Logger('./logs/run_' + str(datetime.datetime.now()))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

weights_matrix = create_matrix(vocab_size, "vocab.txt", model)

criterion = nn.CrossEntropyLoss()
model = BiLSTM(emsize, hidden_dim, vocab_size, num_entities, batch_size, weights_matrix).to(device)
num_words_indexed = 2
num_entities_indexed = 0
vocab_dict = {'|||': 1} # word : index
entity_dict = dict()

def index_strings(strings):
  '''strings is a list of strings'''
  l = []
  global num_words_indexed
  global num_entities_indexed
  for i in strings:
    if i in vocab_dict:
      l.append(vocab_dict[i])
    else:
      vocab_dict[i] = num_words_indexed
      num_words_indexed += 1
    if i.startswith('@entity'):
      if i not in entity_dict:
        entity_dict[i] = num_entities_indexed
        num_entities_indexed += 1
  return l

def train_model(model, train_data):
  global entities
  step = 0
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  test_loader = DataLoader(train_set, batch_size = batch_size, collate_fn=lambda x : x, shuffle=True)
  model = model.to(device)
  loss_criterion = criterion.to(device)
  for epoch in range(num_epochs):
    # (batch_d,batch_q,batch_label)
    data_iter = iter(test_loader)
    for i, thing in enumerate(data_iter):
      sentence_converted = [(index_strings(d), index_strings(q), entity_dict[e]) for d, q, e in thing]
      dq = [torch.Tensor(d + [1] + q).long().to(device) for d, q, e in sentence_converted]
      lengths = torch.Tensor([len(c) for c in dq])
      lengths, idx = lengths.sort(0, descending=True)
      dq = [dq[i] for i in idx]
      expected = torch.Tensor([e for _, _, e in sentence_converted]).flatten().long().to(device)
      expected = expected[idx]
      output = model(dq)
      
      entities = [sorted(list(set([entity_dict[e] for e in d if e.startswith("@entity")]))) for d, _, _ in thing]
      entities = [entities[i] for i in idx]
      
      def calculate_loss(i):
        t = output[i]
        predictions = t[entities[i]]
        index_of_correct = entities[i].index(expected[i].item())
        criterion = nn.CrossEntropyLoss()
        return criterion(predictions.unsqueeze(0), torch.Tensor([index_of_correct]).long().to(device))
      
      loss = sum([calculate_loss(i) for i in range(len(thing))]) / len(thing)
      
      optimizer.zero_grad()
#       loss = loss_criterion(output, expected)
      loss.backward()
      optimizer.step()
      
      logger.scalar_summary("loss", loss.item(), step+1)
      step += 1

train_model(model, test_set)

def test_model(model, test_data):
  total = 0
  correct = 0
  with torch.no_grad():
    test_loader = DataLoader(test_set, batch_size = 1, collate_fn=lambda x : x)
    for i, thing in enumerate(test_loader):
      sentence_converted = [(index_strings(d), index_strings(q), entity_dict[e]) for d, q, e in thing]
      dq = [torch.Tensor(d + [1] + q).long().to(device) for d, q, e in sentence_converted]
      lengths = torch.Tensor([len(c) for c in dq])
      lengths, idx = lengths.sort(0, descending=True)
      dq = [dq[i] for i in idx]
      expected = torch.Tensor([e for _, _, e in sentence_converted]).flatten().long().to(device)
      expected = expected[idx]
      output = model(dq)
      
      entities = [sorted(list(set([entity_dict[e] for e in d if e.startswith("@entity")]))) for d, _, _ in thing]
      entities = [entities[i] for i in idx]
      
      predictions = output[0][entities[0]]
      index_of_correct = entities[0].index(expected[0].item())
      criterion = nn.CrossEntropyLoss()
#       return criterion(predictions.unsqueeze(0), torch.Tensor([index_of_correct]).long().to(device))
      
      _, argmax = torch.max(predictions, 0)
      print(argmax, len(predictions))
      if (argmax.item() == index_of_correct):
        correct += 1
      total += 1
  return correct / total
      
print(test_model(model, test_set))

In [0]:
! export CUDA_LAUNCH_BLOCKING=1; python3 foo.py

In [0]:
def test_model(model, test_data):
  total = 0
  correct = 0
  with torch.no_grad():
    test_loader = DataLoader(test_set, batch_size = 1, collate_fn=lambda x : x)
    for i, thing in enumerate(test_loader):
      sentence_converted = [(index_strings(d), index_strings(q), int(e[7:])) for d, q, e in thing]
      dq = [torch.Tensor(d + [1] + q).long().to(device) for d, q, e in sentence_converted]
      lengths = torch.Tensor([len(c) for c in dq])
      lengths, idx = lengths.sort(0, descending=True)
      dq = [dq[i] for i in idx]
      expected = torch.Tensor([e for _, _, e in sentence_converted]).flatten().long().to(device)
      expected = expected[idx]
      output = model(dq)
      
      entities = [sorted(list(set([int(e[7:]) for e in d if e.startswith("@entity")]))) for d, _, _ in thing]
      entities = [entities[i] for i in idx]
      
      predictions = output[0][entities[0]]
      index_of_correct = entities[0].index(expected[0].item())
      criterion = nn.CrossEntropyLoss()
#       return criterion(predictions.unsqueeze(0), torch.Tensor([index_of_correct]).long().to(device))
      
      _, argmax = torch.max(predictions, 0)
      print(argmax, len(predictions))
      if (argmax.item() == index_of_correct):
        correct += 1
      total += 1
  return correct / total
      
test_model(model, test_set)